In [ ]:
# https://github.com/dennisdv1/Memory-Reduction-Pandas-CSV/blob/master/reducing%20memory.ipynb

In [16]:
import pandas as pd
import numpy as np
import sys

In [97]:
data = pd.read_excel('data.xlsx')

In [99]:
def overview_data (df):
    memory_usage = df.memory_usage(deep=True)
    overview = pd.DataFrame({'mem_usg_MB': round(memory_usage/1024/1024, 2),'data_types': df.dtypes})
    overview = overview.sort_values('mem_usg_MB')
    mb_number = round(overview['mem_usg_MB'].sum(),2)
    print('Total memory use: {} MB'.format(mb_number))
    return overview

In [100]:
overview_data(data)

Total memory use: 18.0 MB


,mem_usg_MB,data_types
Index,0.00,NaN
Datum_neu,0.21,datetime64[ns]
Floating,0.21,float64
Year,0.21,int64
Variable_code,1.59,object
Value,1.61,object
Industry_code_NZSIOC,1.63,object
Industry_aggregation_NZSIOC,1.70,object
Units,1.94,object
Variable_category,2.02,object


In [101]:
def get_memory_reduction(column, as_type):
    
    types = ['int', 'float', 'category']
    
    # Get original memory usage in MB's
    original = data[column].memory_usage(deep=True)/1024/1024
    
    # If any of the types being downsized is int/float/category, get memory usage after reduction and change column type
    if any(i in as_type for i in types):
        after_reduction = data[column].astype(as_type).memory_usage(deep=True)/1024/1024
        data[column] = data[column].astype(as_type)
    
    # if type is changed to datetime64, execute following:
    elif as_type == 'datetime':
        after_reduction = pd.to_datetime(data[column]).memory_usage(deep=True)/1024/1024
        data[column] = pd.to_datetime(data[column])
    
    print("Original memory usage of the column '{}': {} MB".format(column, round(original, 2)))
    print("Memory after downsizing to '{}': {} MB".format(as_type, round(after_reduction, 2)))
    print('Memory reduction of {}%'.format(round((1 - (after_reduction/original)) * 100, 2)))
    print()

In [102]:
def convert_data (df, date):
    
    # Find all non-numeric columns
    non_numeric_col = df.select_dtypes(include=['object'])
    
    # Find all int-columns    
    int_col = df.select_dtypes(include=[np.integer])
    
    # Find all float-columns 
    float_col = df.select_dtypes(include=[np.float])
    
    # Find all date-columns
    date_col = df[date]
    
    #################################################
    ########### Convert all columns #################
    #################################################
    
    for column in non_numeric_col:
        get_memory_reduction(column, 'category')
        
    for column in int_col:
        get_memory_reduction(column, 'int16')
    
    for column in float_col:
        get_memory_reduction(column, 'float16')
        
    for column in date_col:
        if len(date_col) != 0:
            get_memory_reduction(column, 'datetime')
        else:
            continue        

In [103]:
convert_data(data, date = ['Datum_neu'])

Original memory usage of the column 'Industry_aggregation_NZSIOC': 1.7 MB
Memory after downsizing to 'category': 0.03 MB
Memory reduction of 98.42%

Original memory usage of the column 'Industry_code_NZSIOC': 1.63 MB
Memory after downsizing to 'category': 0.07 MB
Memory reduction of 95.93%

Original memory usage of the column 'Industry_name_NZSIOC': 2.43 MB
Memory after downsizing to 'category': 0.04 MB
Memory reduction of 98.28%

Original memory usage of the column 'Units': 1.94 MB
Memory after downsizing to 'category': 0.03 MB
Memory reduction of 98.61%

Original memory usage of the column 'Variable_code': 1.59 MB
Memory after downsizing to 'category': 0.03 MB
Memory reduction of 98.11%

Original memory usage of the column 'Variable_name': 2.1 MB
Memory after downsizing to 'category': 0.03 MB
Memory reduction of 98.52%

Original memory usage of the column 'Variable_category': 2.02 MB
Memory after downsizing to 'category': 0.03 MB
Memory reduction of 98.67%

Original memory usage of t

In [104]:
overview_data(data)

Total memory use: 1.41 MB


,mem_usg_MB,data_types
Index,0.00,NaN
Industry_aggregation_NZSIOC,0.03,category
Units,0.03,category
Variable_category,0.03,category
Variable_code,0.03,category
Variable_name,0.03,category
Industry_code_ANZSIC06,0.04,category
Industry_name_NZSIOC,0.04,category
Floating,0.05,float16
Year,0.05,int16
